### 제주 Airbnb reviews

In [1]:
import json
import re

In [2]:
file_lst = ['translated_jeju.json','translated_seogwipo.json']

In [3]:
review = {}
for file in file_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [4]:
review_en={}

for home_id,rev_lsts in review.items():
    review_en[home_id] = []
    for rev_lst in rev_lsts:
        try:
            if str(rev_lst['language'][-2:]) == 'en':
                review_en[home_id].append([rev_lst['rating'],rev_lst['t_comments']])
        except:
            pass

In [5]:
len(review_en)

9976

### Data Preprocessing Step
##### 1. Tokenization
##### 2. Stop word elimination
##### 3. Stemming : 단어를 기본형으로 바꾸어준다. 복수형은 단수형으로, 과거형은 현재형으로 바꾸는 과정
##### 4. Representation

In [6]:
import nltk
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
import tqdm

### Tokenization

In [7]:
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

100%|█████████████████████████████████████████████████████████████████████████████| 9976/9976 [00:14<00:00, 689.42it/s]


In [8]:
# 길이가 2보다 작은 문장 없애기
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        r = []
        for s in rev[1] : 
            if len(s) > 2 :
                r.append(s)
        rev[1] = r        

100%|███████████████████████████████████████████████████████████████████████████| 9976/9976 [00:00<00:00, 43824.24it/s]


###  Stop words elimination

In [9]:
# 소문자와 대문자가 아닌 것은 공백으로 대체
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        for sentence in rev[1]:
            sentence = re.sub('[^a-zA-Z\s]', '', sentence)

In [10]:
from nltk.corpus import stopwords
stopWords =list(stopwords.words('english'))
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
add_stopwords = ['airbnb','im','youre','hongdae', 'seoul', 'daegu', 'jeju' ,'busan', 'korea','gangneung','itaewon','myeongdong','john','gangnam','also','even','dont','namsan','incheon']
# additional_stopwords = []
stopWords = set(stopWords + add_stopwords)

In [12]:
# 문장별 단어단위 tokenizing 후 pos_tag
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sentence in rev[1]:
            token = nltk.regexp_tokenize(sentence.lower() ,"[\w']+")
            no_stopword_token = [w for w in token if not w in stopWords]
            sent_pos = nltk.pos_tag(no_stopword_token)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

100%|██████████████████████████████████████████████████████████████████████████████| 9976/9976 [55:10<00:00,  3.01it/s]


### 명사, 형용사, 동사, 부사 lemmatization

In [13]:
lemm = WordNetLemmatizer()

def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word, pos = word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n').lower() +'_N')
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a').lower() +'_J')
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v').lower() +'_V')
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r').lower() +'_R')
        else:pass
    return lemm_sent

In [14]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

100%|█████████████████████████████████████████████████████████████████████████████| 9976/9976 [00:14<00:00, 670.53it/s]


In [15]:
with open('review_postag_jeju.json','w',encoding='utf-8') as fp:
        json.dump(review_en,fp)

### LDA 

In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
review = {}
with open('review_postag_jeju.json','r',encoding='utf-8') as fp:
    review.update(json.load(fp))

In [5]:
texts = []
for home_id,rev_lst in review.items():
    for rev in rev_lst:
        for sentence in rev[1] :
            texts.append(sentence)

In [6]:
texts[:2]

[['disappointed_J', 'different_J', 'expect_V', 'picture_N'],
 ['couch_J', 'tv_N', 'furniture_N', 'etc_N']]

In [7]:
dic = corpora.Dictionary(texts)

In [8]:
corpus = [dic.doc2bow(text) for text in texts]

In [9]:
len(corpus)

468090

In [10]:
# corpus: the corpus used to train the topic model.
# id2word: a dictionary that maps word numerical identifiers to word strings 
# alpha: the topic distribution. If we set it to 'auto', then we tell the algorithm to determine the topic proportions automatically.
# num_topics: how many topics the algorithm should find.
# passes: how many times the algorithm should go through the corpus. If you are using a large corpus , then this can be set to 1
# model.num_topics: the number of topics.
# model.show_topic(topic_number, topn) returns the probability distribution over words in one topic. for instance [(0.06, 'food'), (0.04, 'cook'), (0.03, 'fry'), ...]
# model.alpha[topic_number]: the prominence of a topic.

#%%time
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word = dic, passes = 30 ,iterations = 3000,alpha='symmetric', eta='auto') #LDA model 생성

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [11]:
ldamodel.save('lda_jeju.lda')

#### Airnb category : 정확성 / 의사소통 / 청결도 / 위치 / 체크인 / 가치

In [12]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.025*"restaurant_N" + 0.023*"location_N" + 0.020*"convenient_N" + 0.018*"airport_N" + 0.017*"minute_N" + 0.017*"car_N" + 0.016*"station_N" + 0.015*"close_R" + 0.015*"bus_N" + 0.014*"walk_V"'),
 (1,
  '0.058*"room_N" + 0.029*"well_R" + 0.019*"clean_J" + 0.018*"big_J" + 0.016*"people_N" + 0.016*"bathroom_N" + 0.013*"towel_N" + 0.013*"floor_N" + 0.012*"good_J" + 0.012*"bed_N"'),
 (2,
  '0.054*"time_N" + 0.041*"go_V" + 0.038*"stay_V" + 0.034*"trip_N" + 0.032*"good_J" + 0.030*"next_J" + 0.028*"come_V" + 0.027*"day_N" + 0.026*"family_N" + 0.022*"island_N"'),
 (3,
  '0.028*"breakfast_N" + 0.023*"morning_N" + 0.022*"sea_N" + 0.022*"see_V" + 0.019*"view_N" + 0.018*"delicious_J" + 0.018*"house_N" + 0.017*"love_V" + 0.015*"nice_J" + 0.013*"pretty_R"'),
 (4,
  '0.070*"host_N" + 0.049*"kind_N" + 0.028*"thank_N" + 0.016*"give_V" + 0.015*"much_J" + 0.015*"make_V" + 0.014*"help_V" + 0.014*"day_N" + 0.012*"owner_N" + 0.011*"helpful_J"'),
 (5,
  '0.087*"place_N" + 0.050*"nice_J" + 0.042*"clean_

In [3]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_jeju.lda')

In [13]:
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt

In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\asd00\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.057757  0.046721       1        1  18.933716
5      0.211738  0.067125       2        1  18.091206
1      0.202423  0.059847       3        1  17.108648
0     -0.101349 -0.402199       4        1  15.810575
3      0.093972  0.034681       5        1  15.333457
4     -0.349027  0.193825       6        1  14.722397, topic_info=     Category          Freq             Term         Total  loglift  logprob
term                                                                        
49    Default  47685.000000          place_N  47685.000000  30.0000  30.0000
38    Default  31401.000000           host_N  31401.000000  29.0000  29.0000
70    Default  30787.000000           time_N  30787.000000  28.0000  28.0000
69    Default  39915.000000           room_N  39915.000000  27.0000  27.0000
66    Default  21765.000000           kind_N  21765.000000  26.0000  26.0000
110   Default  34461.000000           nice_J  34461.000000  25.0000  25.0000
77    Default  32828.000000          clean_J  32828.000000  24.0000  24.0000
75    Default  21678.000000           stay_V  21678.000000  23.0000  23.0000
241   Default  30079.000000             go_V  30079.000000  22.0000  22.0000
251   Default  19689.000000           trip_N  19689.000000  21.0000  21.0000
102   Default  18281.000000          quiet_J  18281.000000  20.0000  20.0000
55    Default  24638.000000          great_J  24638.000000  19.0000  19.0000
602   Default  17240.000000           next_J  17240.000000  18.0000  18.0000
17    Default  15961.000000           come_V  15961.000000  17.0000  17.0000
425   Default  12960.000000      breakfast_N  12960.000000  16.0000  16.0000
20    Default  21602.000000            day_N  21602.000000  15.0000  15.0000
45    Default  23401.000000           stay_N  23401.000000  14.0000  14.0000
61    Default  12320.000000          thank_N  12320.000000  13.0000  13.0000
87    Default  12063.000000     restaurant_N  12063.000000  12.0000  12.0000
314   Default  17799.000000         family_N  17799.000000  11.0000  11.0000
39    Default  19422.000000       location_N  19422.000000  10.0000  10.0000
94    Default  12246.000000      recommend_V  12246.000000   9.0000   9.0000
370   Default  12355.000000         island_N  12355.000000   8.0000   8.0000
71    Default  22486.000000           well_R  22486.000000   7.0000   7.0000
291   Default  10533.000000        morning_N  10533.000000   6.0000   6.0000
219   Default  11733.000000           want_V  11733.000000   5.0000   5.0000
119   Default  10166.000000            sea_N  10166.000000   4.0000   4.0000
123   Default  10078.000000            see_V  10078.000000   3.0000   3.0000
182   Default   9731.000000     convenient_N   9731.000000   2.0000   2.0000
113   Default  15011.000000           view_N  15011.000000   1.0000   1.0000
...       ...           ...              ...           ...      ...      ...
280    Topic6   4327.723145       friendly_R   4327.871094   1.9158  -4.6362
518    Topic6   4114.028320           late_J   4114.176758   1.9158  -4.6869
194    Topic6   3910.668457       friendly_J   3910.815918   1.9158  -4.7375
344    Topic6   3661.319580    reservation_N   3661.468018   1.9158  -4.8034
334    Topic6   3520.344971          check_N   3520.492676   1.9158  -4.8427
136    Topic6   3360.077148           help_N   3360.224365   1.9158  -4.8893
133    Topic6   3286.656006         always_R   3286.803223   1.9158  -4.9114
977    Topic6   3245.846924       question_N   3245.993652   1.9158  -4.9239
88     Topic6   3240.727783        there's_N   3240.874756   1.9158  -4.9255
392    Topic6   2875.148682         lovely_R   2875.295166   1.9157  -5.0452
1395   Topic6   2577.689209        welcome_V   2577.835205   1.9157  -5.1544
305    Topic6   2530.345215            ask_V   2530.491699   1.9157  -5.1729
727    Topic6   2505.525146        helpful_N   2505.670654  

In [15]:
pyLDAvis.save_html(vis,'LDA_jeju.html')